<a href="https://colab.research.google.com/github/lanpty58/mystudy/blob/main/DuckNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kornia
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install -U numpy==1.21.2

In [ ]:
!pip install numpy --upgrade --ignore-installed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 39.5 MB/s eta 0:00:00


In [ ]:
!pip install -U opencv-python==4.8.0.76

In [ ]:
import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
from keras.callbacks import CSVLogger
from datetime import datetime
import os
import cv2
import logging
import sys
import time

import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, precision_score, recall_score
from kornia.losses import focal_loss

In [ ]:
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
img_size = 448
dataset_type = 'kvasir'
lr = 1e-4
seed_value = 58800
filters = 17
optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
ct = datetime.now()
model_type = 'DuckNet'
progress_path = 'ProgressFull/' + dataset_type + '_progress_csv_' + model_type + '_filters_' + str(filters) +  '_' + str(ct) + '.csv'
progressfull_path = 'ProgressFull/' + dataset_type + '_progress_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.txt'
plot_path = 'ProgressFull/' + dataset_type + '_progress_plot_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.png'
model_path = 'ModelSaveTensorFlow/' + dataset_type + '/' + model_type + '_filters_' + str(filters) + '_' + str(ct)

EPOCHS = 600
min_loss_for_saving = 0.2

In [ ]:
!unzip /content/drive/MyDrive/AI/data/train.zip

Archive:  /content/drive/MyDrive/AI/data/train.zip
replace content/data/train/malignant/malignant (98)_mask.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
#Path
OUTPUT_DIR = '/content/output'
DATASET_DIR = '/content/content/data/train'
CHECKPOINT = None

#Eval
WEIGHT = r''

In [ ]:
def init_path(task):
    weight_dir = os.path.join(OUTPUT_DIR, task)
    os.makedirs(weight_dir, exist_ok=True)
    log_dir = weight_dir
    logger_name = f'{task}'
    return weight_dir, log_dir, logger_name

def setup_logger(logger_name, output_dir):
    import os
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    # create file handler which logs even debug messages
    fh = logging.FileHandler(os.path.join(output_dir, 'log.log'))
    fh.setLevel(logging.DEBUG)
    # create console handler with a higher log level
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s: %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to logger
    logger.addHandler(fh)
    logger.addHandler(ch)
    return logger

In [ ]:
!mkdir output


mkdir: cannot create directory ‘output’: File exists


In [ ]:
import tensorflow as tf
from keras.layers import Conv2D, UpSampling2D, add
from keras.models import Model
from keras.layers import BatchNormalization


kernel_initializer = 'he_uniform'
interpolation = 'nearest'
def double_convolution_with_batch_normalization(x, filters, dilation_rate = 1):
  x = Conv2D(filters, (3, 3), activation = 'relu', kernel_initializer= kernel_initializer, padding = 'same', dilation_rate = dilation_rate)(x)
  x = BatchNormalization(axis = -1)(x)
  x = Conv2D(filters, (3, 3), activation = 'relu', kernel_initializer= kernel_initializer, padding = 'same', dialation_rate = dilation_rate)(x)
  x = BatchNormalization(axis = -1)(x)
  return x

def resnet_conv2D_block(x, filters, dilation_rate = 1):
  x1 = Conv2D(filters, (1, 1), activation='relu', kernel_initializer= kernel_initializer, padding = 'same', dilation_rate = dilation_rate)(x)
  x = Conv2D(filters, (3, 3), activation = 'relu', kernel_initializer=kernel_initializer, padding = 'same', dilation_rate = dilation_rate)(x)
  x = BatchNormalization(axis = -1)(x)
  x = Conv2D(filters, (3, 3), activation = 'relu', kernel_initializer= kernel_initializer, padding = 'same', dilation_rate = dilation_rate)(x)
  x = BatchNormalization(axis = -1)(x)
  x_final = add([x, x1])
  x_final = BatchNormalization(axis=-1)(x_final)
  return x_final

def widescope_conv2D_block(x, filters):
  x = Conv2D(filters, (3, 3), activation ='relu', kernel_initializer= kernel_initializer, padding = 'same', dilation_rate = 1)(x)
  x = BatchNormalization(axis = -1)(x)
  x = Conv2D(filters, (3, 3), activation = 'relu', kernel_initializer= kernel_initializer, padding = 'same', dilation_rate = 2)(x)
  x = BatchNormalization(axis = -1)(x)
  x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding = 'same', dilation_rate = 3)(x)
  x = BatchNormalization(axis = -1)(x)
  return x

def midscope_conv2D_block(x, filters):
  x = Conv2D(filters, (3, 3), activation = 'relu', kernel_initializer = kernel_initializer, padding = 'same', dilation_rate = 1)(x)
  x = BatchNormalization(axis = -1)(x)
  x = Conv2D(filters, (3, 3), activation = 'relu', kernel_initializer = kernel_initializer, padding = 'same', dilation_rate = 2)(x)
  x = BatchNormalization(axis = -1)(x)
  return x

def seperated_conv2D_block(x, filters, size = 3, padding = 'same'):
  x = Conv2D(filters,(1, size), activation = 'relu', kernel_initializer = kernel_initializer, padding = padding)(x)
  x = BatchNormalization(axis = -1)(x)
  x = Conv2D(filters, (size, 1), activation = 'relu', kernel_initializer = kernel_initializer, padding = padding)(x)
  x = BatchNormalization(axis = -1)(x)
  return x

def conv_block_2D(x, filters, block_type, repeat = 1, dilation_rate = 1, size = 3, padding = 'same'):
  result = x
  for i in range(0, repeat):
    if block_type == 'separated':
      result = seperated_conv2D_block(result, filters, size = size, padding = padding, )
    elif block_type == 'duckv2':
      result = duckv2_conv2D_block(result, filters, size = size)
    elif block_type == 'midscope':
      result = midscope_conv2D_block(result, filters)
    elif block_type == 'widescope':
      result = widescope_conv2D_block(result, filters)
    elif block_type == 'resnet':
      result = resnet_conv2D_block(result, filters, dilation_rate)
    elif block_type == 'conv':
      result = Conv2D(filters, (size, size),
                      activation='relu', kernel_initializer=kernel_initializer, padding=padding)(result)
    elif block_type == 'double_convolution':
      result = double_convolution_with_batch_normalization(result, filters, dilation_rate)
    else:
      return None
    return result

def duckv2_conv2D_block(x, filters, size):
  x = BatchNormalization(axis= -1)(x)
  x1 = widescope_conv2D_block(x, filters)
  x2 = midscope_conv2D_block(x, filters)
  x3 = conv_block_2D(x, filters, 'resnet', repeat = 1)
  x4 = conv_block_2D(x, filters, 'resnet', repeat = 2)
  x5 = conv_block_2D(x, filters, 'resnet', repeat = 3)
  x6 = seperated_conv2D_block(x, filters, size = 6, padding = 'same')
  x = add([x1, x2, x3, x4, x5, x6])
  x = BatchNormalization(axis = -1)(x)
  return x

def create_model(img_height, img_width, input_channels, out_classes, starting_filters):
  input_layer = tf.keras.layers.Input((img_height, img_width, input_channels))
  print( 'Starting DUCK-Net')
  p1 = Conv2D(starting_filters * 2, 2, strides=2, padding='same')(input_layer)
  p2 = Conv2D(starting_filters * 4, 2, strides=2, padding='same')(p1)
  p3 = Conv2D(starting_filters * 8, 2, strides=2, padding='same')(p2)
  p4 = Conv2D(starting_filters * 16, 2, strides=2, padding='same')(p3)
  p5 = Conv2D(starting_filters * 32, 2, strides=2, padding='same')(p4)
  t0 = conv_block_2D(input_layer, starting_filters, 'duckv2', repeat=1)

  l1i = Conv2D(starting_filters * 2, 2, strides=2, padding='same')(t0)
  s1 = add([l1i, p1])
  t1 = conv_block_2D(s1, starting_filters * 2, 'duckv2', repeat=1)
  l2i = Conv2D(starting_filters * 4, 2, strides=2, padding='same')(t1)
  s2 = add([l2i, p2])
  t2 = conv_block_2D(s2, starting_filters * 4, 'duckv2', repeat=1)
  l3i = Conv2D(starting_filters * 8, 2, strides=2, padding='same')(t2)
  s3 = add([l3i, p3])
  t3 = conv_block_2D(s3, starting_filters * 8, 'duckv2', repeat=1)

  l4i = Conv2D(starting_filters * 16, 2, strides=2, padding='same')(t3)
  s4 = add([l4i, p4])
  t4 = conv_block_2D(s4, starting_filters * 16, 'duckv2', repeat=1)

  l5i = Conv2D(starting_filters * 32, 2, strides=2, padding='same')(t4)
  s5 = add([l5i, p5])
  t51 = conv_block_2D(s5, starting_filters * 32, 'resnet', repeat=2)
  t53 = conv_block_2D(t51, starting_filters * 16, 'resnet', repeat=2)

  l5o = UpSampling2D((2, 2), interpolation=interpolation)(t53)
  c4 = add([l5o, t4])
  q4 = conv_block_2D(c4, starting_filters * 8, 'duckv2', repeat=1)

  l4o = UpSampling2D((2, 2), interpolation=interpolation)(q4)
  c3 = add([l4o, t3])
  q3 = conv_block_2D(c3, starting_filters * 4, 'duckv2', repeat=1)
  l3o = UpSampling2D((2, 2), interpolation=interpolation)(q3)
  c2 = add([l3o, t2])
  q6 = conv_block_2D(c2, starting_filters * 2, 'duckv2', repeat=1)

  l2o = UpSampling2D((2, 2), interpolation=interpolation)(q6)
  c1 = add([l2o, t1])
  q1 = conv_block_2D(c1, starting_filters, 'duckv2', repeat=1)
  l1o = UpSampling2D((2, 2), interpolation=interpolation)(q1)
  c0 = add([l1o, t0])
  z1 = conv_block_2D(c0, starting_filters, 'duckv2', repeat=1)

  output = Conv2D(out_classes, (1, 1), activation='sigmoid')(z1)

  model = Model(inputs=input_layer, outputs=output)

  return model

In [ ]:
import keras.backend as K
import tensorflow as tf

def dice_metric_loss(ground_truth, predictions, smooth = 1e-6):
  ground_truth = K.cast(ground_truth, tf.float32)
  predictions = K.cast(predictions, tf.float32)
  ground_truth = K.flatten(ground_truth)
  predictions = K.flatten(predictions)
  intersection = K.sum(predictions*ground_truth)
  union = K.sum(predictions) +K.sum(ground_truth)
  dice = (2*intersection +smooth)/(union +smooth)
  return dice

In [ ]:
model = create_model(img_height=img_size, img_width = img_size, input_channels = 3, out_classes = 1, starting_filters = filters)
model.compile(optimizer = optimizer, loss = f1_score)


Starting DUCK-Net


In [ ]:
import keras.backend as K
def calculate_overlap_metrics(pred, gt, eps = 1e-5):

  output = pred.view(-1,)
  target = gt.view(-1,).float()

  tp = K.sum(output * target)  # TP
  fp = K.sum(output * (1 - target))  # FP
  fn = K.sum((1 - output) * target)  # FN
  tn = K.sum((1 - output) * (1 - target))  # TN

  # pixel_acc = (tp + tn + eps) / (tp + tn + fp + fn + eps)
  dice = (2 * tp + eps) / (2 * tp + fp + fn + eps)
  iou = ( tp + eps) / ( tp + fp + fn + eps)
  precision = (tp + eps) / (tp + fp + eps)
  recall = (tp + eps) / (tp + fn + eps)
  #specificity = (tn + eps) / (tn + fp + eps)

  return iou, dice, precision, recall

In [ ]:
import numpy as np
from PIL import Image
from skimage.io import imread
from tqdm import tqdm
def split_dataset(dataset_dir):
  benign, malignant, normal = [], [], []
  benign_images = [os.path.join(dataset_dir, 'benign', file) for file in os.listdir(os.path.join(dataset_dir, 'benign')) if file.endswith('.png')]
  malignant_images = [os.path.join(dataset_dir, 'malignant', file) for file in os.listdir(os.path.join(dataset_dir, 'malignant')) if file.endswith('png')]
  normal_images = [os.path.join(dataset_dir, 'normal', file) for file in os.listdir(os.path.join(dataset_dir, 'normal')) if file.endswith('.png')]

  images, masks = [], []
  image_paths = benign_images + malignant_images + normal_images
  dataset_size = len(image_paths)
  X = np.zeros((dataset_size, img_size, img_size, 3), dtype=np.float32)
  Y = np.zeros((dataset_size, img_size, img_size), dtype=np.float32)

  for i in range(len(image_paths)):
        if '_mask' in image_paths[i]:
          image_path = image_paths[i].replace('_mask.png', '.png')
          # image = imread(image_path)
          # mask = imread(image_paths[i])

          # pillow_image = Image.fromarray(image)
          # pillow_image = pillow_image.resize((img_size, img_size))
          # image = np.array(pillow_image)

          image = Image.open(image_path)
          image = image.resize((img_size, img_size))
          mask = Image.open(image_paths[i]).convert('L')
          mask = mask.resize((img_size, img_size))

          # Convert mask to NumPy array and initialize bi_mask
          mask = np.array(mask)
          bi_mask = np.zeros_like(mask)

          # pillow_mask = Image.fromarray(mask)
          # pillow_mask = pillow_mask.resize((img_size, img_size), resample=Image.LANCZOS)

          # mask = np.array(pillow_mask)

          # bi_mask = np.zeros((img_size, img_size), dtype=np.bool_)
          # for r in range(img_size):
          #   for c in range(img_size):
          #       if mask[r, c] >= 127:
          #           bi_mask[r, c] = 1
          bi_mask = np.where(mask < 127, bi_mask, 1)
          Y[i] = np.array(bi_mask)
          X[i] = np.array(image)/255

          # images.append(image)
          # masks.append(mask)

  Y = np.expand_dims(Y, axis=-1)

  train_image, val_image, train_mask, val_mask = train_test_split(X, Y, test_size = 0.1, shuffle= True, random_state = 42)
  return train_image, val_image, train_mask, val_mask


In [ ]:
train_images, val_images, train_masks, val_masks = split_dataset(DATASET_DIR)


In [ ]:
step = 0
N_FOLDS = 1
BATCH_SIZE = 8
NUM_WORKERS = 0

def train(train_images, val_images, train_masks, val_masks):
  TASK = 'Segmentation'
  weight_dir, log_dir, logger_name = init_path(TASK)
  logger = setup_logger(logger_name, log_dir)
  best_f1 = 0
  stale = 0
  start_epoch = 1

  for epoch in range(start_epoch, EPOCHS+1):
    train_images, train_masks = augment_images(train_images, train_masks)
    csv_logger = CSVLogger(progress_path, append=True, separator=';')
    model.fit(x = train_images, y = train_masks, epochs = 1, batch_size=BATCH_SIZE, validation_data=(val_images, val_masks), verbose=1, callbacks=[csv_logger])
    prediction_valid = model.predict(val_images, verbose = 0)
    loss_valid = dice_metric_loss(val_masks, prediction_valid)
    loss_valid = loss_valid.numpy()
    print("Loss Validation: "+ str(loss_valid))
    with open(progress_path, 'a') as f:
      f.write('epoch: ' + str(epoch) + '\nval_loss: ' + str(loss_valid) + '\n\n\n')

    if min_loss_for_saving > loss_valid:
      min_loss_for_saving = loss_valid
      print('Saved model with val_loss: ', loss_valid)
      model.save(model_path)

      gc.collect()

In [ ]:
train_images.shape


(1121, 448, 448, 3)

In [ ]:

import albumentations as albu
aug_train = albu.Compose([
                  albu.HorizontalFlip(),
                  albu.VerticalFlip(),
                  albu.ColorJitter(brightness=(0.6, 1.6),contrast=0.2, saturation = 0.1, hue = 0.01, always_apply=True),
                  albu.Affine(scale = (0.5, 1.5), translate_percent=(-0.125, 0.125), rotate = (-180, 180), shear = (-22.5, 22), always_apply=True)])

def augment_images(train_images, train_masks):
    x_train_out = []
    y_train_out = []
    # train_images = train_images.astype(np.uint8)
    # train_masks = train_masks.astype(np.uint8)


    for i in range (len(train_images)):


        ug = aug_train(image=train_images[i], mask=train_masks[i])
        x_train_out.append(ug['image'])
        y_train_out.append(ug['mask'])

    return np.array(x_train_out), np.array(y_train_out)

In [ ]:
train(train_images, val_images, train_masks, val_masks)

In [ ]:
print("Loading the model")
model = tf.keras.models.load_model(model_path, custom_objects={'dice_metric_loss':dice_metric_loss})
prediction_train = model.predict(train_images, batch_size=BATCH_SIZE)
prediction_valid = model.predict(val_images, batch_size=BATCH_SIZE)


print("Predictions done")

dice_train = f1_score(np.ndarray.flatten(np.array(train_masks, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
dice_valid = f1_score(np.ndarray.flatten(np.array(val_masks, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Dice finished")


miou_train = jaccard_score(np.ndarray.flatten(np.array(train_masks, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))

miou_valid = jaccard_score(np.ndarray.flatten(np.array(val_masks, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Miou finished")


precision_train = precision_score(np.ndarray.flatten(np.array(train_masks, dtype=bool)),
                                  np.ndarray.flatten(prediction_train > 0.5))

precision_valid = precision_score(np.ndarray.flatten(np.array(val_masks, dtype=bool)),
                                  np.ndarray.flatten(prediction_valid > 0.5))

print("Precision finished")


recall_train = recall_score(np.ndarray.flatten(np.array(train_masks, dtype=bool)),
                            np.ndarray.flatten(prediction_train > 0.5))

recall_valid = recall_score(np.ndarray.flatten(np.array(val_masks, dtype=bool)),
                            np.ndarray.flatten(prediction_valid > 0.5))

print("Recall finished")


accuracy_train = accuracy_score(np.ndarray.flatten(np.array(train_masks, dtype=bool)),
                                np.ndarray.flatten(prediction_train > 0.5))

accuracy_valid = accuracy_score(np.ndarray.flatten(np.array(val_masks, dtype=bool)),
                                np.ndarray.flatten(prediction_valid > 0.5))


print("Accuracy finished")


final_file = 'results_' + model_type + '_' + str(filters) + '_' + dataset_type + '.txt'
print(final_file)

with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write('dice_train: ' + str(dice_train) + ' dice_valid: ' + str(dice_valid) + '\n\n')
    f.write('miou_train: ' + str(miou_train) + ' miou_valid: ' + str(miou_valid) + '\n\n')
    f.write('precision_train: ' + str(precision_train) + ' precision_valid: ' + str(precision_valid)  + '\n\n')
    f.write('recall_train: ' + str(recall_train) + ' recall_valid: ' + str(recall_valid)  + '\n\n')
    f.write('accuracy_train: ' + str(accuracy_train) + ' accuracy_valid: ' + str(accuracy_valid)  + '\n\n\n\n')

print('File done')

Loading the model


OSError: No file or directory found at ModelSaveTensorFlow/kvasir/DuckNet_filters_17_2024-02-22 15:20:13.630923